In [1]:
import pandas as pd
import openpyxl
from itables import show


## Goal: Get Spend Category into PO Data

In [26]:
# Inital Load of Data no longer needed
# df_po_data = pd.read_excel('data/Open_PO_Spend_large.xlsx', engine='calamine')
# df_sourcing_spend = pd.read_csv('data/spend_data_large_sourcing.csv', engine='pyarrow')
# Load data to parquet files
# df_po_data.to_parquet('data/Open_PO_Spend_large.parquet')
# df_sourcing_spend.to_parquet('data/spend_data_large_sourcing.parquet')


## Load files, and show the first 20 rows

In [5]:
df_po_data = pd.read_parquet('data/Open_PO_Spend_large.parquet')
df_sourcing_spend = pd.read_parquet('data/spend_data_large_sourcing.parquet')

In [6]:
show(df_po_data)

Loading ITables v2.5.2 from the internet... (need help?)


# Columns usefull from Purchase Order Data(df_po_data)
1. PO Number
2. PO Line Item Number
3. PO Item Description
4. Ariba Requisition Title Local
5. Vendor Name
6. Vendor Number


In [7]:
show(df_sourcing_spend)

Loading ITables v2.5.2 from the internet... (need help?)


# Columns usefull from Spend Data(df_sourcing_spend)
1. Spend_Data_PO_Number
2. Spend_Data_PO_Line_Item_Number
3. Spend_Data_Vendor_Number
4. Spend_Data_Invoice_Type
5. Spend_Data_Line_Item_Text - Is this the same as PO line item description?
6. Spend_Data_Vendor_Name
7. Spend_Data_Sourcing_Category

## Explore the PO numbers in both datasets
1. Lenth of both dataset 
2. How many rows have a PO number that is Non-PO
3. Percent of PO numbers in df_po_data that ARE in df_sourcing_spend



In [23]:
# how many PO numbers match
print(f"Length of df_po_data: {len(df_po_data)}")
print(f"Length of df_sourcing_spend: {len(df_sourcing_spend)}\n")

#in both how many rows are there that have a PO number that is Non-PO

print(f"Number of rows with non PO PO number in df_sourcing_spend: {len(df_sourcing_spend[df_sourcing_spend['Spend_Data_PO_Number'].str.contains('Non-PO')])}\n\n")



# Percent of PO numbers in df_po_data that ARE in df_sourcing_spend
pct_found = df_po_data['PO Number'].isin(df_sourcing_spend['Spend_Data_PO_Number']).sum() / len(df_po_data) * 100
print(f"Percent of PO numbers in df_po_data that ARE in df_sourcing_spend: {pct_found:.2f}%")


Length of df_po_data: 628577
Length of df_sourcing_spend: 698411

Number of rows with non PO PO number in df_sourcing_spend: 405216


Percent of PO numbers in df_po_data that ARE in df_sourcing_spend: 59.08%


## Check Consistency of Sourcing Category, in Spend Data
1. Percent of Vendor Numbers with more than one category
2. Percent of PO Numbers with more than one category
3. Percent of (Vendor Number, PO Number) combinations with more than one category



In [18]:
# Analytics: Check category consistency across different key combinations

# 1. Percent of Vendor Numbers with more than one category
vendor_category_counts = df_sourcing_spend.groupby('Spend_Data_Vendor_Number')['Spend_Data_Sourcing_Category'].nunique()
vendors_multi_cat = (vendor_category_counts > 1).sum()
vendors_total = len(vendor_category_counts)
pct_vendors_multi_cat = vendors_multi_cat / vendors_total * 100

print("=" * 60)
print("CATEGORY CONSISTENCY ANALYSIS")
print("=" * 60)
print(f"\n1. VENDOR NUMBER:")
print(f"   Total unique vendors: {vendors_total:,}")
print(f"   Vendors with >1 category: {vendors_multi_cat:,}")
print(f"   Percent with multiple categories: {pct_vendors_multi_cat:.2f}%")
print(f"   Percent with single category: {100 - pct_vendors_multi_cat:.2f}%")

# 2. Percent of PO Numbers with more than one category
po_category_counts = df_sourcing_spend.groupby('Spend_Data_PO_Number')['Spend_Data_Sourcing_Category'].nunique()
pos_multi_cat = (po_category_counts > 1).sum()
pos_total = len(po_category_counts)
pct_pos_multi_cat = pos_multi_cat / pos_total * 100

print(f"\n2. PO NUMBER:")
print(f"   Total unique PO numbers: {pos_total:,}")
print(f"   POs with >1 category: {pos_multi_cat:,}")
print(f"   Percent with multiple categories: {pct_pos_multi_cat:.2f}%")
print(f"   Percent with single category: {100 - pct_pos_multi_cat:.2f}%")

# 3. Percent of (Vendor Number, PO Number) combinations with more than one category
vendor_po_category_counts = df_sourcing_spend.groupby(
    ['Spend_Data_Vendor_Number', 'Spend_Data_PO_Number']
)['Spend_Data_Sourcing_Category'].nunique()
vendor_po_multi_cat = (vendor_po_category_counts > 1).sum()
vendor_po_total = len(vendor_po_category_counts)
pct_vendor_po_multi_cat = vendor_po_multi_cat / vendor_po_total * 100

print(f"\n3. (VENDOR NUMBER, PO NUMBER) COMBINATION:")
print(f"   Total unique combinations: {vendor_po_total:,}")
print(f"   Combinations with >1 category: {vendor_po_multi_cat:,}")
print(f"   Percent with multiple categories: {pct_vendor_po_multi_cat:.2f}%")
print(f"   Percent with single category: {100 - pct_vendor_po_multi_cat:.2f}%")

# Summary comparison
print("\n" + "=" * 60)
print("SUMMARY: Which key has the best category consistency?")
print("=" * 60)
print(f"   Vendor Number only:        {100 - pct_vendors_multi_cat:.2f}% single category")
print(f"   PO Number only:            {100 - pct_pos_multi_cat:.2f}% single category")
print(f"   (Vendor, PO) combination:  {100 - pct_vendor_po_multi_cat:.2f}% single category")

CATEGORY CONSISTENCY ANALYSIS

1. VENDOR NUMBER:
   Total unique vendors: 8,205
   Vendors with >1 category: 4,725
   Percent with multiple categories: 57.59%
   Percent with single category: 42.41%

2. PO NUMBER:
   Total unique PO numbers: 111,986
   POs with >1 category: 4,665
   Percent with multiple categories: 4.17%
   Percent with single category: 95.83%

3. (VENDOR NUMBER, PO NUMBER) COMBINATION:
   Total unique combinations: 116,760
   Combinations with >1 category: 4,665
   Percent with multiple categories: 4.00%
   Percent with single category: 96.00%

SUMMARY: Which key has the best category consistency?
   Vendor Number only:        42.41% single category
   PO Number only:            95.83% single category
   (Vendor, PO) combination:  96.00% single category


In [12]:
#create Lookup table from PO data with PO number and Spend Category
# 
# # Ensure both join keys are strings
df_po_data['PO Number'] = df_po_data['PO Number'].astype(str)
df_po_data['Vendor Number'] = df_po_data['Vendor Number'].astype(str)  # adjust column name as needed

df_sourcing_spend['Spend_Data_PO_Number'] = df_sourcing_spend['Spend_Data_PO_Number'].astype(str)
df_sourcing_spend['Spend_Data_Vendor_Number'] = df_sourcing_spend['Spend_Data_Vendor_Number'].astype(str)

# Create a lookup table: get ONE category per (PO, Vendor) combination
# Using mode (most common) or first occurrence
category_lookup = (
    df_sourcing_spend
    .groupby(['Spend_Data_Vendor_Number'])['Spend_Data_Sourcing_Category']
    .first()  # or use .agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0])
    .reset_index()
)

print(f"Lookup table size: {len(category_lookup)}")


Lookup table size: 8205


In [13]:
# Now merge - this will be a many-to-one join (preserves df_po_data row count)
df_po_data_with_category = df_po_data.merge(
    category_lookup,
    left_on=['Vendor Number'],  # adjust column names to match your df_po_data
    right_on=['Spend_Data_Vendor_Number'],
    how='left'
)

print(f"Result shape: {df_po_data_with_category.shape}")  # Should be (628577, 68 or 69)

Result shape: (628577, 68)


In [14]:
#percent of null rows in spend_data_with_category
print(f"Percent of null rows in spend_data_with_category: {df_po_data_with_category['Spend_Data_Sourcing_Category'].isnull().mean().round(5)*100}%")





Percent of null rows in spend_data_with_category: 5.638%


In [11]:
# Check for null values in 'Spend_Data_Posting_Date_Fiscal_Year' column
num_null_fiscal_year = df_sourcing_spend['Spend_Data_Posting_Date_Fiscal_Year'].isnull().sum()
print(f"Number of null values in 'Spend_Data_Posting_Date_Fiscal_Year': {num_null_fiscal_year}")

# Optionally, check for null values in all columns
null_counts_all_cols = df_sourcing_spend.isnull().sum()
print("Null value count per column:")
print(null_counts_all_cols)


Number of null values in 'Spend_Data_Posting_Date_Fiscal_Year': 0
Null value count per column:
Spend_Data_Posting_Date_Fiscal_Year                     0
Spend_Data_Posting_Date_Fiscal_Year_and_Quarter         0
Spend_Data_Posting_Date                                 0
Spend_Data_Vendor_Invoice_Date                          0
Spend_Data_PO_Number                                    0
Spend_Data_PO_Line_Item_Number                     438801
Spend_Data_Commodity_Code                          430949
Spend_Data_Commodity_Description_SpendMart         442053
Spend_Data_Vendor_Number                                0
Spend_Data_Invoice_Type                                 0
Spend_Data_Company_Code                                 0
Spend_Data_Country                                      0
Spend_Data_Document_Type_Group                          0
Spend_Data_Accounting_Document_Number                   0
Spend_Data_Accounting_Document_Type                     0
Spend_Data_GL_Account_Number       

In [20]:
# Get unique values from 'Spend_Data_Sourcing_Portfolio'
unique_portfolios = df_sourcing_spend['Spend_Data_Sourcing_Portfolio'].unique()
print("Unique values in 'Spend_Data_Sourcing_Portfolio':")
print(unique_portfolios)

# Get unique values from 'Spend_Data_Posting_Date_Fiscal_Year'
unique_fiscal_years = df_sourcing_spend['Spend_Data_Posting_Date_Fiscal_Year'].unique()
print("Unique values in 'Spend_Data_Posting_Date_Fiscal_Year':")
print(unique_fiscal_years)


Unique values in 'Spend_Data_Sourcing_Portfolio':
['Professional Services' 'Marketing' 'Non-addressable' 'Technology'
 'Human Resources' 'Facilities' 'Travel']
Unique values in 'Spend_Data_Posting_Date_Fiscal_Year':
[2023 2024 2025 2026]


In [27]:
target_categories = ['Technology']
target_fiscal_years = [2025, 2026]

filtered_sourcing_spend = df_sourcing_spend[
    (df_sourcing_spend['Spend_Data_Sourcing_Portfolio'].isin(target_categories)) &
    (df_sourcing_spend['Spend_Data_Posting_Date_Fiscal_Year'].isin(target_fiscal_years))
].copy()

show(filtered_sourcing_spend.head(20))


Loading ITables v2.5.2 from the internet... (need help?)


In [33]:
row_count = len(filtered_sourcing_spend)
print(f"Number of rows in filtered_sourcing_spend: {row_count}")

Number of rows in filtered_sourcing_spend: 34652


#context for AI. 
1. Spend_Data_Line_Item_Text
2. Spend_Data_Commodity_Description
3. Spend_Data_Vendor_Name

#other info I care about
1. Spend_Data_Posting_Date_Fiscal_Year
2. Spend_Data_Sourcing_Portfolio
3. Spend_Data_Sourcing_Sub_Portfolio
4. Spend_Data_Sourcing_Category
5. Spend_Data_Sourcing_Sub_Category



In [39]:

# Count tokens for columns 1-4 of df_sourcing_spend using all rows.
# We assume a standard tokenization where each word, number, or punctuation mark may count as one token.
# Here, let's count total tokens roughly as the number of words split by whitespace in the selected columns, for all rows.

import numpy as np

# Select columns 1-4 (Python zero-based index: columns 0,1,2,3)
cols_to_count = filtered_sourcing_spend.columns[0:4]
selected_data = filtered_sourcing_spend[cols_to_count].astype(str)

def count_tokens(s):
    # A rough token count: split by whitespace
    return len(str(s).split())

# Apply token count to each cell, then sum over all cells
token_counts = selected_data.applymap(count_tokens)
total_tokens = int(np.sum(token_counts.values))

print(f"Estimated total tokens for columns {list(cols_to_count)} over all rows \n\nTotal tokens: {total_tokens}")


Estimated total tokens for columns ['Spend_Data_Posting_Date_Fiscal_Year', 'Spend_Data_Posting_Date_Fiscal_Year_and_Quarter', 'Spend_Data_Posting_Date', 'Spend_Data_Vendor_Invoice_Date'] over all rows 

Total tokens: 381172


/var/folders/25/d6q11ppn1w35d_v79_lyc50m0000gp/T/ipykernel_10888/4154744202.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  token_counts = selected_data.applymap(count_tokens)


In [40]:
#create a signature column that is a combination of the line item text, commodity description, and vendor name
filtered_sourcing_spend['signature'] = filtered_sourcing_spend['Spend_Data_Line_Item_Text'] + ' ' + filtered_sourcing_spend['Spend_Data_Commodity_Description'] + ' ' + filtered_sourcing_spend['Spend_Data_Vendor_Name']

In [42]:
unique_signatures = filtered_sourcing_spend['signature'].unique()
print(f"Number of unique signatures: {len(unique_signatures)}")

#print number of rows orgininally
print(f"Number of rows originally: {len(filtered_sourcing_spend)}")





Number of unique signatures: 12421
Number of rows originally: 34652


In [1]:
#check to see how many PO values match in both dataframes
print(f"Number of PO values that match in both dataframes: {len(df_po_data['PO_Number'].isin(df_sourcing_spend['Spend_Data_PO_Number']))}")

#check to see how many PO values match in both dataframes
print(f"Number of PO values that match in both dataframes: {len(df_po_data['PO_Number'].isin(df_sourcing_spend['Spend_Data_PO_Number']))}")



NameError: name 'df_po_data' is not defined